In [2]:
import torch
import numpy as np
import os
import csv
import pandas as pd

In [ ]:
label_path = '/data/wsi/TCTGC50k-labels/TCTGC50k-volume1-labels-path.csv'
data_path = '/data/wsi/TCTGC10k-features/rtdetr-v3-2025.1.7/000100042.pt'

df = pd.read_csv(label_path)
wsi = torch.load(data_path)

In [ ]:
pt_file = './153070575-1-1.pt'

pt = torch.load(pt_file)

In [ ]:
print(pt)

In [6]:
all_data = pt['patch_load']

In [ ]:
print(type(all_data[0]))

In [ ]:
x = torch.rand(5,2,225)
print(x[:,0].shape)

In [ ]:
x = torch.tensor([-1e9, -1e9, -1e9])

x = x.softmax(dim=-1)

print(x)

In [1]:
import torch
import numpy as np
from models.model_v1 import *
from models.vision_transformer_wsi import *

/home/ligaojie/anaconda3/envs/biomed/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
model = MaskedAttentionPoolLatent(
    in_dims=256,
    num_heads=8
)

In [3]:
model = CellAggregator(
    in_dims=256,
    num_heads=8
)

In [2]:
model = Model_V1(
    in_dims=256,
    num_classes=9,
    depth=1,
    num_heads=8,
    proj_drop=0.02,
    attn_drop=0.02,
    drop_path=0.01
)

In [3]:
mask = torch.randint(low=0, high=2, size=(2, 800, 150))
print(mask)

x = torch.rand(2, 800, 150, 256)

tensor([[[0, 0, 0,  ..., 0, 1, 0],
         [0, 1, 0,  ..., 0, 0, 1],
         [1, 0, 0,  ..., 0, 1, 0],
         ...,
         [0, 1, 1,  ..., 1, 1, 0],
         [1, 1, 0,  ..., 0, 0, 0],
         [0, 0, 1,  ..., 1, 1, 1]],

        [[1, 1, 0,  ..., 0, 1, 1],
         [0, 0, 0,  ..., 0, 1, 0],
         [0, 1, 1,  ..., 0, 1, 1],
         ...,
         [1, 1, 0,  ..., 0, 1, 0],
         [1, 0, 0,  ..., 1, 0, 1],
         [1, 0, 1,  ..., 0, 1, 1]]])


In [5]:
device = torch.device('cuda:0')

In [6]:
model.to(device)
print(next(model.parameters()).device)
x = x.to(device)
mask = mask.to(device)

# logit = model(x, mask)

cuda:0


In [7]:
logit = model(x, mask)

In [8]:
print(logit.shape)

torch.Size([2, 9])


In [ ]:
y = torch.randint(low=0, high=2, size=(2, 10)).to(device)

cri = AsymmetricCriterion(num_classes=10)

dic = cri(logit, y)

print(dic)

In [ ]:
0.3881+0.4623+0.3458+0.0329+0.3183+0.0271+0.3244+0.3876+0.3091+0.2882

In [ ]:
print(sum(dic[k] for k in dic.keys()))

In [ ]:
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
a = [torch.rand(4,256), torch.rand(7,256), torch.rand(3,256), torch.rand(5,256)]
padded_patches = pad_sequence(a, batch_first=True, padding_value=0)


print(padded_patches.shape)

In [ ]:
lb = torch.tensor(1)
print(F.one_hot(lb, num_classes=9).shape)

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

In [10]:
total_params = count_parameters(model)

# 将参数数量转换为以 "M" 为单位
params_in_millions = total_params / 1_000_000
formatted_params = f"{params_in_millions:.2f}M"  # 保留两位小数

print(f"Total number of parameters: {formatted_params}")

Total number of parameters: 4.22M
